In [55]:
!pip uninstall kaggle -y
!pip uninstall --upgrade.pip
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6

Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: --upgrade.pip
  Using cached kaggle-1.5.6-py3-none-any.whl


In [56]:
!kaggle competitions download -c 2021-ai-w3-p2

2021-ai-w3-p2.zip: Skipping, found more recently modified local copy (use --force to force download)


In [57]:
!unzip 2021-ai-w3-p2.zip

Archive:  2021-ai-w3-p2.zip
replace sample_submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [71]:
import pandas as pd
import numpy as np

train=pd.read_csv("train_data.csv")
test=pd.read_csv("test_data.csv")
submit=pd.read_csv("sample_submit.csv")

In [72]:
test

,ID,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize
0,0,2,Ka+,2017,1,10150,4,145,57.7,1.2
1,1,2,Kuga,2013,1,50216,0,145,53.3,2.0
2,2,3,Q5,2016,1,34189,0,200,47.9,2.0
3,3,1,I10,2020,1,1900,4,145,56.5,1.0
4,4,0,3 Series,2014,0,79809,0,20,68.9,2.0
...,...,...,...,...,...,...,...,...,...,...
16826,16826,2,Grand Tourneo Connect,2016,0,29000,0,125,56.6,1.5
16827,16827,4,Yaris,2019,1,2812,4,145,47.9,1.5
16828,16828,2,Kuga,2017,1,46800,0,150,54.3,2.0
16829,16829,3,A7,2017,3,14562,0,145,52.3,3.0


In [73]:
x=train.drop(["ID","price"],axis=1)
test=test.drop(["ID"],axis=1)
y=train["price"]

In [74]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(x["model"])

x["model"]=le.transform(x["model"])
test["model"]=le.transform(test["model"])
x_model_re=le.inverse_transform(x["model"])

In [75]:
 from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x)

x_std=scaler.transform(x)
test_std=scaler.transform(test)

In [76]:
x["year"]=x_std[:,2]
test["year"]=test_std[:,2]
x["mileage"]=x_std[:,4]
test["mileage"]=test_std[:,4]
x["tax"]=x_std[:,6]
test["tax"]=test_std[:,6]

In [77]:
import torch
import random
seed=777
random.seed(777)
torch.manual_seed(seed)
device='cuda' if torch.cuda.is_available() else 'cpu'
if device=='cuda':
  torch.cuda.manual_seed_all(seed)

In [78]:
import torch
x=np.array(x)
y=np.array(y)
test=np.array(test)
x=torch.FloatTensor(x)
y=torch.FloatTensor(y)
test=torch.FloatTensor(test)

In [79]:
linear1=torch.nn.Linear(9,128)
linear2=torch.nn.Linear(128,256)
linear3=torch.nn.Linear(256,128)
linear4=torch.nn.Linear(128,1)


relu=torch.nn.ReLU()
dropout=torch.nn.Dropout(0.1)

In [80]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)

Parameter containing:
tensor([[-0.2090,  0.0153,  0.1558, -0.2023,  0.0022, -0.0658,  0.1244,  0.1227,
         -0.0996, -0.2036, -0.0149,  0.0817, -0.0533,  0.0320, -0.1787, -0.1676,
          0.1320, -0.1222, -0.0065,  0.0253, -0.1849,  0.0357, -0.1274,  0.0125,
         -0.0358,  0.0119,  0.2052,  0.1454, -0.0017,  0.1467,  0.0279, -0.1177,
         -0.1610,  0.0065, -0.1404, -0.1115, -0.1797, -0.0729,  0.0807,  0.0006,
         -0.0316, -0.0381, -0.2143, -0.1191,  0.0441,  0.1188,  0.1391, -0.0571,
          0.0849, -0.1895,  0.0847, -0.1619, -0.0043, -0.1496,  0.0319,  0.0760,
         -0.0096, -0.0690,  0.0796, -0.1493, -0.0660,  0.1194, -0.1827, -0.1484,
          0.0728,  0.0185,  0.0886, -0.0021,  0.0961,  0.2020, -0.0327, -0.1827,
          0.1346,  0.1911, -0.1174,  0.1060, -0.1562, -0.0535,  0.1059, -0.0648,
         -0.1645,  0.1110, -0.1304,  0.1104,  0.2121,  0.0036, -0.0929,  0.0116,
          0.1100,  0.2008, -0.1269,  0.0867,  0.0562,  0.1171,  0.1208,  0.0467,
      

In [81]:
model=torch.nn.Sequential(linear1,relu,dropout,linear2,relu,dropout,linear3,relu,dropout,linear4).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
loss=torch.nn.MSELoss().to(device)

In [82]:
for stop in range(1000):
  optimizer.zero_grad()
  h=model(x.to(device))
  cost=loss(h,y.unsqueeze(1).to(device))
  cost.backward()
  optimizer.step()
  if stop%10==0:
    print(stop,cost.item())

0 383160256.0
10 146141584.0
20 146703392.0
30 118881952.0
40 104671480.0
50 92768040.0
60 74949608.0
70 57797204.0
80 46366880.0
90 40389500.0
100 34525172.0
110 31191166.0
120 27809186.0
130 25097116.0
140 23190148.0
150 22029268.0
160 21242678.0
170 20628596.0
180 20323104.0
190 19733068.0
200 19505052.0
210 19144514.0
220 18891236.0
230 19084232.0
240 18821126.0
250 18604214.0
260 18470322.0
270 18534092.0
280 18659376.0
290 18343226.0
300 17761276.0
310 18023858.0
320 17462856.0
330 17530282.0
340 17535424.0
350 17561022.0
360 17348494.0
370 17227226.0
380 16899566.0
390 16728032.0
400 16458055.0
410 16327425.0
420 16098822.0
430 16129873.0
440 16258490.0
450 15661462.0
460 15611946.0
470 15729015.0
480 15598047.0
490 15167900.0
500 15393020.0
510 15035985.0
520 15445661.0
530 14985062.0
540 15022313.0
550 15306716.0
560 14641605.0
570 14741875.0
580 14790822.0
590 14538635.0
600 14492642.0
610 14613157.0
620 14533742.0
630 14399989.0
640 14245765.0
650 14610004.0
660 14280269.0
6

In [83]:
pred=list()
with torch.no_grad():
  model.eval()
  y_test=model(test)
  a,b=torch.max(y_test,dim=1)
  pred.extend(a.tolist())

In [84]:
submit["price"]=pred

In [85]:
submit["price"].max()

83053.5859375

In [86]:
submit.to_csv("submit.csv",index=False)
!kaggle competitions submit -c 2021-ai-w3-p2 -f submit.csv -m "Message"

100% 337k/337k [00:03<00:00, 108kB/s]
Successfully submitted to 자동차 가격 예측